In [1]:
_gmv_keys = [
    'temperature.output', 
    'temperature.man', 
    'temperature.interlocked', 
    'temperature.pv', 
    'temperature.sp', 
    'temperature.error', 
    'temperature.mode', 
    'agitation.output', 
    'agitation.man', 
    'agitation.interlocked', 
    'agitation.pv', 
    'agitation.sp', 
    'agitation.error', 
    'agitation.mode', 
    'ph.manUp',
    'ph.outputDown',
    'ph.error',
    'ph.pv',
    'ph.sp',
    'ph.manDown',
    'ph.outputUp',
    'ph.mode',
    'ph.interlocked',
    'do.manUp',
    'do.outputDown',
    'do.error',
    'do.pv',
    'do.sp',
    'do.manDown',
    'do.outputUp',
    'do.mode',
    'do.interlocked',
    'maingas.error',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'maingas.interlocked',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'condenser.output',
    'condenser.man',
    'condenser.pv',
    'condenser.sp', 
    'condenser.error', 
    'condenser.mode',
    'pressure.error',
    'pressure.pv',
    'level.error',
    'level.pv'
]

In [65]:
import threading
from hello.hello3 import open_hello
from datetime import datetime
import time


class _HelloLoggerTask(threading.Thread):
    def __init__(self, ip="", tag=None, vars=(), poll_interval=2, start_time=None):
        
        # Config
        self.ip = ip
        self._vars = self._parse_vars(vars)
        self._tag = tag
        self.poll_interval = 5
        
        # Runtime
        # self.data_lock = threading.Lock()
        self._data = {}
        self._h = None
        self._should_stop = False
        self._last_timestamp = None
        self._start_time = start_time
        
    def stop(self):
        self._should_stop = True
        
    def run(self):
        
        # create some locals for brevity
        
        self._h = h = open_hello(ip)
        call_hello = self._call_hello

        if self._tag is None:
            self._tag = call_hello(h.reactorname).replace(".", "_")
        
        if self._start_time is None:
            self._start_time = datetime.now()
        start_time = self._start_time
            
        # unpack some values so they can't be modified 
        # at runtime by anyone evil or just dumb (hopefully)
        
        tag = self._tag
        recorded = self._vars
        
        # strings that don't need re-computing
        last_request_string = name + ".Last Request.Time"
        elapsed_time_string = name + ".Elapsed Time.hr"
            
        while not self._should_stop:
            
            # An existential question - when did the query actually occur?
            # Did it occur as soon as the request was sent, after it was
            # recieved, or halfway between the two? The most likely scenario
            # for a delayed response is that our server software is choking 
            # for some reason, so use the timestamp immediately after return
            # as the best estimate. 
            
            next_run = time.time() + self.poll_interval
            mv = call_hello(h.gpmv)
            t = datetime.now()
            
            data = [
                (last_request_string, t),
                (elapsed_time_string, (start_time - t).total_seconds() / 3600)
            ]
            
            for key, k1, k2 in recorded:
                v = mv[k1][k2]
                data.append((key, v))
                self.data = data
                
            # Sleep chokes on negative numbers
            left = next_run - time.time()
            if left > 0:
                time.sleep(left)
        
    def _parse_vars(self, vars):
        seen = set()
        keys = []
        for k in vars:
            k = k.lower()
            if k in seen:
                raise ValueError("Duplicate key found: %r"%k)
            if k not in _gmv_vars:
                raise ValueError("Invalid key: %r"%k)
            seen.add(k)
            try:
                k1, k2 = k.split(".")
            except ValueError:  # no ".", bad key
                raise ValueError("%r is an invalid key" % k) from None
            keys.append((k1, k2))
        return tuple(keys)
        
    def _call_hello(f, *args, **kw):
        while True:
            try:
                return f(*args, **kw)
            except NotLoggedInError:
                self._h.login()
            
            # XXX Handle connection errors
        
class HelloMultiLogger():
    def __init__(self, filename, vars, sample_interval=10):
        self._tasks = []
        self._should_stop = False
        self._filename = filename
        self._vars = vars
        self._data_log = DataLog()
        self._sample_interval = sample_interval
        
    def stop(self):
        self._should_stop = True
        for t in self._tasks:
            t.stop()
        
    def add_logger(self, ip, tag, poll_interval=2, start=False, start_time=None):
        if self.is_alive():
            raise ValueError("Can't add tasks to already-running data logger.")
        task = _HelloLoggerTask(ip, tag, self._vars, poll_interval)
        if start:
            task.start()
        self._tasks.append(task)
        return task
    
    def start(self):
        for t in self._tasks:
            if not t.is_alive():
                t.start()
        super().start()
        
    def run(self):
        
        while True:
            
            # I went back on forth on the best way to get the data off
            # each thread when it was time to log. The format of each data
            # object is a list of tuples, and the assignment of the instance
            # attribute is atomic. Therefore, just grabbing the data object 
            # without bothering with event flags or locks is fine, assuming CPython. 
            
            ts = datetime.now()
            data = [t.data for t in self._tasks]
            self._data_log.add_record(ts, data)
            time.sleep(self._sample_interval)
            
            
                
from collections import defaultdict
                    
                    
class DataLog():
    def __init__(self):
        self.n = 0
        self.timestamps = []
        self.data = defaultdict(lambda: [""] * self.n)
        
    def add_record(self, ts, data):
        self.timestamps.append(ts)
        for rd in data:
            for key, value in rd:
                rl = self.get_record_list(key)
                rl.append(value)
        self.n += 1
        
    def get_record_list(self, key):
        rl = self.data[key]
        if len(rl) < self.n:
            rl.extend("" for _ in range(self.n - len(rl)))
        return rl
            

In [ ]:
vars_i_care_about = [
    'agitation.pv',
    'temperature.pv'
]

ml = HelloMultiLogger("Test.csv")

In [66]:
dl = DataLog()

In [70]:
dl.timestamps.extend(range(5)); dl.n += 5

In [72]:
dl.timestamps

[0, 1, 2, 3, 4, 0, 1, 2, 3, 4]

In [71]:
dl.get_record_list("foo")

['', '', '', '', '', '', '', '', '', '']